# Transformers

이전 장에서는 Transformer 아키텍처가 두각을 나타낼 때까지 NLP에서 유행하는 모델링 아키텍처인 RNN을 다루었습니다.

트랜스포머는 최신 NLP의 핵심입니다. 2017년에 처음 제안된 원래 아키텍처는 (딥 러닝) 세계를 폭풍으로 몰아넣었습니다. 그 이후로 NLP 문학은 세서미 스트리트 문자(Sesame street characters) 또는 "-former"로 끝나는 단어로 광범위하게 분류되는 모든 종류의 새로운 아키텍처로 범람했습니다.

이 장에서 우리는 바로 그 아키텍처, 즉 트랜스포머를 자세히 살펴볼 것입니다. 우리는 핵심 혁신(the core innovations)을 분석하고 신경망 계층의 새로운 범주인 주의 집중 메커니즘(the attention mechanism)을 탐구할 것입니다.

## Building a Transformer from Scratch

2장과 3장에서는 변환기를 실제로 사용하는 방법과 사전 훈련된 변환기를 활용하여 복잡한 NLP 문제를 해결하는 방법을 살펴보았습니다. 이제 아키텍처 자체에 대해 자세히 알아보고 변환기가 첫 번째 원칙에서 작동하는 방식을 알아봅니다.

"제1원칙"은(는) 무슨 뜻인가요? 음, 우선 Hugging Face Transformers 라이브러리를 사용할 수 없음을 의미합니다. 우리는 이미 이 책에서 그것에 대해 많이 열광했습니다. 그래서 우리는 그것에서 잠시 휴식을 취하고 내부에서 실제로 어떻게 작동하는지 볼 때입니다. 이 장에서는 원시 PyTorch를 대신 사용할 것입니다.

대부분의 연구자가 사용하는 본격적인 딥 러닝 라이브러리인 PyTorch는 자연스럽게 Hugging Face 라이브러리와 마찬가지로 매우 인기 있는 변환기 아키텍처를 구현합니다. 그러나 nn.Module로 노출되는 이 버전은 DIY에 훨씬 더 가깝고 데이터 로더, 옵티마이저 등과 같은 다른 친숙한 PyTorch 도구와 함께 사용하기 위한 것입니다.

앞에서 언급했듯이 딥 러닝 관련 클래스/함수가 수행하는 작업을 확인하는 가장 좋은 방법 중 하나는 유형 서명(type signature)과 입력 및 출력의 차원을 살펴보는 것입니다. 그렇게 해봅시다:

In [ ]:
import torch

In [ ]:
model = torch.nn.Transformer()
model.encoder.layers[0]

In [ ]:
TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm(512,), eps=1e-0.5, elementwise_affine=True)
    (norm2): LayerNorm(512,), eps=1e-0.5, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
)    

처음에는 이것에서 빼낼 것이 너무 많지 않은 것 같습니다. 우리를 위해 정의된 표준 forward() 함수가 있는 상당히 표준적인 PyTorch nn.Module입니다. 원칙적으로 우리는 그것을 훈련 파이프라인에 연결하고 계속할 수 있습니다. 사실 이것은 본질적으로 우리가 2장에서 했던 것입니다. 하지만 이 모듈의 정확한 구성 요소를 이해하려고 노력합시다.

특히 흥미로운 것은 MultiHeadAttention 계층입니다. Dropout, Linear 및 LayerNorm과 같은 대부분의 다른 레이어는 비변환기 모델에서도 볼 수 있을 것으로 예상할 수 있습니다. PyTorch에 의한 변환기의 이 특정 구현(추가 구성 매개변수 없음)은 공교롭게도 제목이 "Attention Is All You Need"인 원본 논문(그림 7-1 참조)의 아키텍처입니다.

요컨대, 이 Transformer 클래스의 가장 중요한 구성 요소는 MultiHeadAttention 계층이라고 해도 무방합니다. 따라서 그것이 무엇이며 어떻게 작동하는지 이해하는 데 시간이 걸리는 것이 합리적입니다.

## Attention Mechanisms

주의 집중(attetion) 메커니즘은 심층 신경망의 계층입니다. 여전히 해석의 여지가 있는 작업이지만 장거리 "전역" 기능을 학습하는 것입니다. 어텐션 메커니즘은 임베딩 벡터의 입력 시퀀스에서 단일 출력 벡터에 기여하는 구성 요소를 결정하는 "정보 라우터"라고 부르는 역할을 합니다. 이 아이디어는 우리가 실제로 세부 사항을 작업하면서 더 명확해질 것입니다.

우리는 작년에 NeurIPS에 참석한 다른 수천 명의 사람들과 마찬가지로 관심(attention)에 대해 이야기하게 되어 기쁩니다. 하지만 그 전에 주목해야 할 중요한 주제는 관련된 작업의 계산적 복잡성이라는 점을 언급해야 합니다. 표시되고 관련된 텐서의 크기에 대해 내적/행렬 곱셈의 수를 생각해 보세요.

### Dot Product Attention

좋아요, 엄밀히 말해서 우리는 이러한 유형의 주의 집중(attention)이 실제 네트워크에 실제로 적용되는 것을 본 적이 없다고 생각합니다. Scaled dot product Attention은 일반적으로 다음에 논의할 Multi-Head Self-Attention의 구성 요소로 언급됩니다.

이국적인 관심(attention) 메커니즘의 세계에서 당신이 해야 할 가장 중요한 질문은 이것입니다: 당신은 사물들 사이의 유사성을 정확히 어떻게 측정합니까? 선형 대수와 GPU의 버킷 로드에 가려진 이 핵심 아이디어는 오늘날 NLP에서 신경망의 기본 동작을 주도하는 것입니다.

그리고 스케일링된 내적은 유사성을 측정하는 가장 단순하고 직관적인 방법 중 하나인 내적을 사용합니다.

당신은 이것에 익숙해야 하지만 간단히 요약해 봅시다. 내적은 두 벡터를 취해 요소별로 곱한 다음 결과를 더하는 연산입니다. 이것은 우리가 "내적"하는 두 벡터가 유사한 구성 요소를 가지고 있으면 해당 요소의 곱이 크고 그 반대의 경우도 마찬가지이기 때문에 유사성을 측정합니다(유사하지 않은 구성 요소가 있는 벡터는 내적이 작다는 점에서).

하지만 진짜 질문은 우리가 내적을 취하는 것이 정확히 무엇입니까? 이 질문에 답하기 위해 이러한 어텐션 메커니즘이 변환기에서 구현되는 방식에 초점을 맞추겠습니다(그림 7-1 참조).

![image](images/transformer_layer.png)

일반적인 변환기는 일련의 단어 벡터를 입력으로 사용하고 각 레이어에서 이를 숨겨진 표현/상태라고 하는 벡터의 다른 시퀀스로 변환합니다(아니요, 그렇게 이름이 붙은 것 같지는 않습니다).

따라서 네트워크의 각 숨겨진 레이어에는 "참석(attent)"하려는 벡터 시퀀스가 있습니다. 그림 7-2를 참조하십시오.

![image](images/dotproduct_1-768x412.png)

자, 여기서 중요한 부분이 있으니 집중하세요(매우 의도된 말장난).

우리가 할 일은 숨겨진 상태 벡터 각각을 3개의 완전히 독립된 벡터(쿼리, 키 및 값)로 변환하는 것입니다.

간단한 행렬 곱셈을 통해 이 변환을 수행하고 이러한 벡터의 차원은 우리에게 달려 있습니다. 유일한 제한은 쿼리 벡터와 키 벡터가 동일한 차원을 가져야 한다는 것입니다(둘 사이에 내적을 수행할 것이므로).:

그런 다음 모든 키 벡터와 마찬가지로 각 시간 단계에서 쿼리 벡터의 내적을 취하여 주의 가중치를 계산하고 그 결과를 softmax합니다. 모든 시간 단계에서 동시에 이 작업을 수행하려면 곱셈을 병렬로 수행하는 행렬에 이러한 벡터를 압축하는 것이 더 효율적입니다. 최종 계산은 다음과 같습니다.:

하지만 그게 우리가 할 수 있는 전부는 아닙니다. 각 쿼리 벡터는 독립적이므로 시간에 따라 병렬화할 수 있습니다.:

하지만 왜? 그것이 트랜스포머 페이퍼를 읽을 때 우리가 스스로에게 물었던 질문입니다. 세 개의 벡터로 나누는 것은 약간 임의적이고 복잡해 보입니다. 예를 들어, 왜 두 개나 네 개가 아닌가?

이름만 보고 직감이 데이터베이스에 뿌리를 두고 있는 것 같습니다. 이것이 신경망 없이 일반적인 오래된 Python 사전(dictionaries)에서 어떻게 작동하는지 생각해 보세요.

대규모 키-값 쌍 시퀀스가 있습니다. 그것이 사전 구조입니다. 다음과 같이 보일 수 있습니다.:

```
sentence = {
    "word_1": "Squirtle"
    "word_2": "is"
    "word_3": "the"
    "word_4": "greatest"
    "word_5": "Pokemon"
    "word_6": "ever"
}
```

이제 나는 당신이 무슨 생각을 하고 있는지 압니다. "푸-리스! 우리 모두는 Squirtle이 Charizard에 대항할 기회가 조금도 없다는 것을 알고 있습니다."

글쎄요, 우리는 다를 것을 간청합니다. 받아 들여.

사전에서 값을 얻으려면 다음과 같은 쿼리를 사용합니다.:

```
a = sentence['word_3']
```

그리고 뒤에서 파이썬이 하는 일은 word_3 쿼리를 문장 사전에 있는 모든 가능한 키와 비교하는 것입니다. 그런 다음 값을 반환하고 변수에 저장합니다.

우리가 dot product Attention으로 하고 있는 것은 비슷합니다. 쿼리 벡터는 추상적인 의미에서 현재 단어가 찾고 있는 것을 나타냅니다. 각 단어 종류와 관련된 키는 각 단어가 제공해야 하는 것을 나타냅니다. 값 벡터에는 쿼리 벡터가 찾고 있던 정보가 포함됩니다. 그러나 우리는 그것이 매우 추상적으로 들린다는 것을 알고 있으므로 예를 보여 드리겠습니다.

다음 문장을 고려하십시오.:

>Mario is short, but the he can jump super high.

이제 변환기가 현재 "he"라는 단어에 대해 작업 중이고 이를 네트워크의 다음 계층으로 전파하려고 한다고 가정해 보겠습니다. 여기서 쿼리 벡터는 대명사 "he"가 정확히 무엇을 가리키는지 명확히 하기 위해 이름이나 사람을 찾는 것일 수 있습니다. 따라서 변환기는 "he"에 대한 쿼리 벡터를 사용하고 문장에서 다른 모든 단어의 키 벡터를 사용하여 이 쿼리 벡터의 내적을 계산합니다. 이러한 각 내적은 쿼리와 키가 일치하는 정도를 측정하는 일종의 정렬 소스를 생성합니다.

이렇게 하면 "Mario"에 해당하는 키 벡터가 켜지고 가장 큰 정렬 점수가 생성됩니다. 이것은 네트워크에 흥미로운 일이 진행되고 있음을 나타내며 네트워크는 주의를 기울여야 합니다(우리가 거기서 무엇을 했는지 보십시오).

그러나 작업은 아직 완료되지 않았습니다. 변환기가 "he"와 문장의 다른 단어 사이의 모든 정렬 점수를 계산하면 이 점수를 softmax를 통해 전달하여 멋진 분포를 생성합니다. 점수를 더 자연스럽게 해석할 수 있습니다. 0은 단어 사이에 연관성이 거의 없다는 것을 의미하고 1은 거의 완벽하게 일치한다는 의미입니다.

각 단어에는 관련 값 벡터가 있으며, 이 벡터는 Python 사전의 값과 마찬가지로 세상의 실제로 의미 있는 콘텐츠를 나타내는 것으로 간주됩니다. 그러나 Python 사전과 달리 각 쿼리는 단일 결과를 반환하지 않습니다. 대신 변환기는 각 단어에 대해 계산한 정규화된 정렬 점수를 가져와 모든 값 벡터에 대해 가중 합계를 수행하는 데 사용합니다. 그 이유는 다소 간단합니다. 예를 들어 우리가 작업하고 있던 문장은 "Mario와 Luigi는 키가 작지만 매우 높이 뛸 수 있습니다."였습니다. 여기서 "그들"에 대한 쿼리는 단일 단어를 찾는 것이 아니라 이 그룹에 맞는 가능한 모든 사람을 찾는 것입니다.

이제 정렬 점수 분포를 생성하면 문장의 다른 부분을 다른 양으로 선택할 수 있습니다! 약간 단순화하면 "Mario" 및 "Luigi"라는 단어의 정규화된 정렬 점수가 0.5이고 다른 모든 단어의 경우 0이라고 상상할 수 있습니다.

변환기는 이제 문장에 참석하고(진지하게도 단어인가요?) 특정 단어(이 예에서는 "he" 및 "they")에 대한 벡터를 생성했습니다. 큰 틀에서 이 단어와 관련이 있습니다.

이제 문장의 모든 단어에 대해 이 프로세스를 반복하여 네트워크의 더 깊은 레이어로 전달할 또 다른 벡터 시퀀스를 얻습니다.

트랜스포머의 인코더 부분에서 이른바 "셀프 어텐션"을 계산할 때 다음과 같은 숨겨진 상태가 계산에 사용됩니다.:

이들은 모두 인코더의 해당 계층에 있는 시퀀스에서 가져옵니다. 디코더의 self-attention도 마찬가지입니다.

쿼리에 대한 디코더 숨겨진 표현과 키 및 값에 대한 인코더 숨겨진 표현을 사용하는 디코더에 사용되는 주의 집중(attention) 계층이 있습니다. 이를 통해 디코더는 이전의 모든 인코더 숨겨진 표현을 처리할 수 있으며 이는 기계 번역과 같은 작업에 유용합니다. 프랑스어 번역가가 실제로 전체 문장을 먼저 영어로 읽지 않고 횡설수설을 토해내는 것을 원하지 않을 것입니다.

그림 7-2에서 self-attention 계층을 시각화하고 그림 7-1에서 전체 계층을 함께 시각화할 수 있습니다. 이 레이어 몇 개를 다른 레이어 위에 쌓으면 붐! 당신은 (거의) 변압기(transformer)를 얻었습니다.

### Scaled Dot Product Attention

하지만 여기에는 약간의 문제가 있습니다. 내적은 정말 빠르고 훌륭하지만 벡터의 크기가 클 때 내적은 상당히 커질 수 있습니다.

우리가 의미하는 바를 보려면 두 개의 랜덤 벡터를 고려하십시오. 하지만 그것에 대해 이야기하는 대신 NumPy의 실제 계산을 보여드리겠습니다.:

In [ ]:
import numpy as np

In [ ]:
small_dots = [
    np.dot(np.random.randn(10),
           np.random.randn(10))
    for i in range(100)]
np.mean(np.absolute(small_dots))

방금 한 것은 크기가 10인 두 개의 임의 배열을 생성하고 이들의 내적을 취하는 것입니다. 확실히 하기 위해 우리는 이것을 100번 반복하고 내적의 평균 크기를 계산하여 임의의 이상값을 얻지 않도록 했습니다.

그래서 값은 약 2.74였습니다. 어떻게 유용합니까? 크기가 10,000인 배열로 동일한 작업을 시도해 봅시다.:

In [ ]:
large_dots = [np.dot(np.random.randn(10000),
                     np.random.randn(10000))
              for i in range(100)]
np.mean(np.absolute(large_dots))

좋아요. 훨씬 더 큽니다. 하지만 생각해보세요. 정렬을 측정하기 위해 내적을 사용하고 있기 때문에 여기서 분명히 잘못된 것이 있습니다. 두 경우 모두 순전히 랜덤 벡터를 생성했으므로 이상적으로는 정렬 점수가 비슷해야 합니다.

그러나 구성 요소는 평균이 0이고 분산이 1인 표준 정규 분포에서 선택되기 때문에 n차원 벡터의 분산은 n입니다(구성 요소의 분산을 더하여 그의 분산을 얻습니다. pedantic, 그것은 벡터의 공분산 행렬의 흔적이지만 너무 긴 이름입니다).

이를 수정하고 모든 차원의 벡터가 거의 동일한 정렬 점수를 갖도록 하기 위해 통계에서 단위 분산으로 정규화하는 방법과 유사하게 이전 주의 집중(attention) 메커니즘을 확장합니다.

새롭고 수정된 어텐션 메커니즘은 다음과 같습니다.:

### Multi Head Self Attention

흥미로운 점은 다음과 같습니다.: 방금 논의한 두 가지 어텐션 메커니즘과 지금 보여주려는 메커니즘은 모두 "Attention Is All You Need"(일명 트랜스포머 페이퍼)라는 동일한 논문에서 나왔습니다. 꽤 멋지죠?

어쨌든, 우리가 할 수 있는 다음 일은 어텐션 메커니즘을 더 작은 어텐션 메커니즘("s" 포함)으로 분할하는 것입니다. 왜 이렇게 하려고 할까요? 근거를 설명하는 좋은 방법은 인기 있는 주의력 테스트 비디오를 사용하는 것입니다. (https://youtu.be/vJG698U2Mvo)

이제 당신은 아마 그 비디오를 전에 본 적이 있을 것입니다(안 본 적이 있다면 놀랍습니다!). 그리고 당신은 첫 패스에서 고릴라를 발견하는 것이 왜 그렇게 어려운지 알 것입니다. 한 번에 한 가지에 주의를 기울이는 것이 더 쉽고 합리적입니다 . 이 경우 비디오에서 찾도록 요구하는 것이 농구 패스입니다. 대신 고릴라를 찾으라는 요청을 받았다면 아마도 고릴라를 찾는 것이 더 쉬웠을 것입니다.

주의 메커니즘은 같은 방식으로 작동합니다. 앞서 논의한 대명사("Mario는 키가 작지만 그는 매우 높이 뛸 수 있습니다")와 같이 언어에서 주의를 기울이고 추적해야 할 많은 항목이 있지만 주인공이 어디로 가는지와 같은 다른 항목도 있습니다. 물리적 공간에서("Mario는 꽃가게에 갔다가 체육관에 가서 스쿼트 50회를 했습니다")

모든 작업을 수행하는 쿼리, 키 및 값 세트가 너무 많을 수 있으며 아마도 당신이 그랬던 것처럼 가끔 고릴라를 놓칠 수 있습니다.

다중 헤드 어텐션 메커니즘은 단일 패스에서 동일한 시퀀스에 어텐션 메커니즘을 여러 번 독립적으로 적용하여 이 문제를 해결하려고 합니다. 고릴라 비디오의 관점에서 이것은 친구가 비디오를 보는 것과 같습니다. 한 사람은 패스에 주의를 기울이고 다른 사람은 고릴라를 찾아 전반적인 주의력을 높일 수 있습니다.

결정적으로, 쿼리 키와 값 매트릭스는 달라야 합니다. 그렇지 않으면 전체 주의 사항을 여러 번 다시 실행하는 것은 계산 낭비일 뿐입니다(패스를 찾는 동안 친구에게 패스를 찾도록 요청하는 것).

쿼리, 키 및 값의 다양성을 생성하기 위해 변환기 네트워크는 단순히 여러 개의 개별 가중치 매트릭스를 사용하여 입력을 여러 쿼리, 키 및 값으로 변환합니다.:

여기서 n은 사용자가 설정한 매개변수이며 이를 헤드의 수라고 합니다. 동일한 시퀀스에서 얼마나 많은 다른 어텐션 계산이 수행되고 있는지 나타냅니다. 고릴라 비디오를 함께 보도록 초대한 사람의 수라고 생각하면 됩니다.

당신은 아직 그 비유에 지쳤습니까? 걱정하지 마세요. 거의 끝났습니다.

이러한 각 "헤드"는 스케일링된 내적 어텐션 계산을 독립적으로(그리고 결정적으로 병렬로) 수행합니다.:

이 모든 숫자 처리가 끝나면 각 어텐션 헤드의 출력에 해당하는 시퀀스의 지점당 n개의 다른 출력 벡터가 남게 됩니다. 그러나 다음 레이어에는 벡터 시퀀스(및 n 벡터 시퀀스)가 필요하므로 변환기는 여러 어텐션 헤드의 출력을 연결하고 다른 학습된 선형 변환을 통해 전달하여 차원이 올바르게 작동하도록 합니다.:

이러한 새로 연결되고 변환된 z 벡터의 시퀀스는 변환기의 다음 계층으로 전달됩니다.

그것은 확실히 한 번에 받아들여야 할 선형 대수학의 양이 많으므로 천천히 다시 검토하여 실제로 이해하는지 확인하십시오. 특히 Multi-Head Self-Attention을 시각화하는 것이 어떻게 작동하는지 이해하는 가장 좋은 방법일 것입니다. Jay Alammar는 이에 대한 훌륭한 기사를 보유하고 있으며(https://jalammar.github.io/illustrated-gpt2/) 거기에 제시된 시각화를 살펴보시기 바랍니다.

### Adaptive Attention Span

자, 드디어 (상대적으로) 새롭고 멋진 작업으로 넘어가겠습니다. 2019년에 Facebook AI Research의 멋진 사람들이 정말 멋진 질문을 했습니다. 변환기를 사용하여 주의해야 할 사항을 학습할 수 있다면 어떨까요?

하지만 이미 트랜스포머가 하는 일이 아닌가요? 이것이 주의 집중(attention) 메커니즘의 전체 요점이 아닙니까?

네. 그러나 우리가 언급하지 않은 또 다른 매우 중요한 사항인 계산 비용이 있습니다. 주의 집중(attention) 메커니즘을 추가하는 것은 저렴하지 않습니다. 배치/문장에 $n$개의 단어가 있는 경우 시퀀스의 모든 토큰에서 각 어텐션 가중치를 계산하는 데 $n^2$ 내적(레이어당)이 필요합니다. 이것은 $n$ 쿼리 벡터 각각과 $n$ 키 각각의 내적을 취해야 하기 때문입니다.

알다시피, 이것은 꽤 빨리 터질 수 있습니다. 예를 들어 일괄 처리/문장에 50개의 가져오기/단어가 있는 경우 계산할 최소 $50^2 = 2,500$ 내적이 있습니다. 그러나 단순히 토큰 수를 2개 증가시켜 52개로 만들면 $52^2 = 2704$ 이상의 내적을 계산할 수 있습니다. 이는 배치당 2개의 추가 토큰을 추가하는 것만으로도 약 200개의 도트 제품(dot products)이 추가되는 것입니다.

물론 매번 모든 단일 토큰에 대해 주의를 계산해야 하는지 의문을 가질 수 있습니다. 조금 과하다고 봅니다. 특히 문자 수준 또는 하위 단어 수준 모델에서 일부 주의 헤드는 단순히 마지막 몇 개의 토큰을 보고 문자 또는 하위 단어를 함께 단어로 맞추려고 시도할 수 있습니다. 그러나 모든 사람들이 마지막 몇 개의 토큰만 보도록 만드십시오.

우리(또는 이 경우 Facebook 팀, 우리는 단지 그들의 작업의 이점을 거두고 있습니다)가 균형을 맞추는 방식은 일부 헤드가 더 큰 토큰 세트에 참석하고 일부 헤드가 마지막 몇 개의 토큰에만 참석하도록 하는 것입니다.

여기에서 소개할 용어가 하나 있습니다: 참석 범위. 이것은 단순히 모델이 처리하는 이전 토큰 수를 나타냅니다. 따라서 헤드의 주의 범위가 5인 경우 헤드가 시퀀스의 현재 위치에서 마지막 5개 토큰에 대해 주의 집중(attention) 메커니즘을 실행한다는 의미입니다.

그렇다면 각 헤드의 주의 집중(attention) 시간을 어떻게 결정할까요? 일반적으로 여기에는 실험, 플롯, 약간의 손 흔들기 및 약간의 추측이 포함됩니다. 그러나 적응형 주의 집중(attention) 범위가 정말 멋진 이유는 각 머리가 훈련 과정을 통해 자체 주의 집중(attention) 범위를 학습할 수 있다는 것입니다!

이 아이디어는 하이퍼파라미터, 즉 참석할 토큰 수를 가져와서 역전파를 통해 자동으로 조정할 수 있는 간단한 매개변수로 만들기 때문에 정말 멋집니다.

당면한 주요 문제는 다음과 같습니다. 주의 공간이라고도 하는 각 헤드가 보는 토큰의 수는 정수이므로 구별할 수 없습니다. 미분할 수 없다는 것은 훈련을 통해 해당 매개변수를 실제로 배울 수 없다는 것을 의미합니다. 그래서 연구팀은 주의 지속 시간의 미분 가능한 버전을 얻을 수 있는 영리한 방법을 찾아야 했습니다.

그들은 토큰 사이의 거리를 취하고 0과 1 사이의 값을 출력하는 마스킹 함수라는 것을 만들어 이를 수행했습니다. 논문에서 그들은 마스킹 함수를 다음과 같이 정의합니다.:

Which we guess looks a little weird. But the plot is actually pretty clear and simple, as shown in Figure 7-3.

따라서 여기서 직감은 두 토큰 사이의 거리 x가 충분히 크면 $m_z(x)$의 값이 0이 된다는 것입니다. 즉, 두 토큰 사이에서 주의 집중(attention) 계산을 수행하지 않는다는 의미입니다.

이 $m_z(x)$ 함수는 매끄럽기 때문에 기울기를 얻을 수 있고 각 어텐션 헤드에 대해 $z$의 값을 조정할 수 있습니다. 더 큰 $z$를 사용하면 관심 헤드는 더 많은 토큰을 살펴보고 그 반대도 마찬가지입니다. $R$은 플롯에 표시되는 램프 섹션의 평활도를 제어하는 하이퍼파라미터입니다.

그러나 가장 중요한 것은 적응형 주의 범위 변환기가 매우 멋진 결과를 가져온다는 것입니다. 다른 변환기보다 상당히 적은 메모리와 FLOPs를 사용하여 enwik8 데이터 세트에서 최첨단 성능을 달성합니다.

### Persistent Memory/All-Attention

self-attention 메커니즘에 대한 이러한 수정은 딥 러닝 연구가 거의 수행하지 않는 단순성에 초점을 맞추기 때문에 약간 흥미롭습니다.

FAIR의 논문에서 소개된 all-attention 계층(예, 다시 같은 사람들)은 성능을 크게 향상시키거나 계산 비용을 줄이지 않습니다. 대신 원래 Transformer 아키텍처에서 다단계 프로세스를 취하여 어텐션 메커니즘만 포함하는 단일 단계로 재공식화합니다.

원래 구현에서 변환기는 각 계층에서 위치별 피드 포워드 네트워크를 사용합니다. 이것이 의미하는 바는 어텐션 메커니즘을 실행한 후 변환기가 다음 레이어의 어텐션 메커니즘으로 전달하기 전에 작은 바닐라 신경망을 통해 시퀀스의 각 벡터를 전달한다는 것입니다.

영구 메모리 문서에 따르면 트랜스포머의 매개변수 대부분은 어텐션 및 셀프 어텐션 메커니즘이 아니라 이러한 피드포워드 네트워크에서 사용됩니다.

그래서 그들의 아이디어는 위치별 피드포워드 네트워크를 완전히 제거하는 것이었습니다. 반드시 매개변수의 수를 줄이기 위한 것은 아니지만(결국 더 많은 매개변수를 다시 추가하게 되기 때문입니다), 단지 그 이유 때문입니다.

그들은 위치별 피드포워드 네트워크의 계산을 응시하면 주의 메커니즘이 수행하는 계산과 실제로 유사해 보인다는 것을 보여주었습니다. 저자가 의미하는 바를 살펴 보겠습니다.:

여기서 피드포워드 네트워크의 $U, V$는 가중치 행렬입니다.

둘 사이의 연결이 실제로 보이지 않습니까? 그래, 우리도 마찬가지야. 그러나 편향 항을 제거하고 ReLU를 softmax로 교체할 때 어떤 일이 발생하는지 살펴보십시오.:

이제 마지막 단계를 자세히 살펴보면 $U$(행렬)과 $Vx_t$(벡터) 사이의 행렬-벡터 곱임을 알 수 있습니다. 그리고 어떻게 작동하는지에 대한 세부 사항을 기억합니다. 이것은 기본적으로 $U$ 행렬 열의 가중 합을 취합니다.:

여기서 주의 집중(attention) 가중치 $a_ti$는 $Vx_t$ 제품에서 계산되고 $u_i$는 $U$의 i번째 열입니다.

이러한 방식으로 계산을 보면 $x_t, V, U$는 확장된 내적 주의 집중(attention)의 쿼리, 키 및 값과 유사합니다.

이 모든 수학의 요점은 무엇입니까? 글쎄, 사실 별로. 죄송합니다.

이 논문의 주요 결론은 피드포워드 네트워크가 수행하는 계산이 매우 유사하기 때문에(사실 베이스 항과 활성화 함수를 무시하면 거의 동일함) 이들을 교체하고 트랜스포머 아키텍처를 만들 수 있을 것입니다. 간단합니다.

우리의 의견으로는 "이봐, 우리는 어텐션 메커니즘 이후에 피드 포워드 네트워크를 제거하고 대신 어텐션을 많이 사용했습니다. 꽤 잘 작동했습니다."라고 말하는 것도 똑같이 유효했을 것입니다. 하지만 이봐, 그게 뭐야.

소중한 자유 시간의 마지막 5분을 방금 우리가 거의 새로운 일을 하지 않는다고 말한 많은 방정식에 대해 머리를 숙이는 데 소비하면서 삶의 의미에 의문을 갖기 시작했다면 두려워하지 마십시오. 왜냐하면 이 논문에는 또 다른 정말 멋진 아이디어가 있었기 때문입니다. 영구 메모리입니다.

피드포워드 네트워크를 어텐션으로 교체하면 모델의 매개변수 수가 줄어들 것이라는 점을 고려하여 작성자는 GPU 메모리가 너무 지루해지지 않도록 호의적으로 결정하여 Nvidia 홈 히터의 온도를 높일 수 있는 몇 가지 새로운 방법을 찾았습니다. 

물론 이제 모델에 더 많은 매개변수를 추가하고 싶다면 레이어 수를 늘리고 컨텍스트 크기를 늘리는 등의 간단한 작업을 항상 수행할 수 있습니다. 하지만 대신 이 FAIR 팀은 매우 영리한 작업을 수행하기로 결정했습니다. 그들은 모델에 독립적인 메모리 뱅크를 제공하기로 결정했습니다.

구체적으로 알려드리겠습니다. "메모리 뱅크"라고 하면 키-값 벡터 쌍의 대규모 컬렉션을 의미합니다. 이러한 키-값 벡터 쌍을 원하는 만큼 가질 수 있으며 실제 교육 데이터와 완전히 독립적입니다.

이렇게 큰 벡터 뱅크가 있으면 텍스트 데이터의 시퀀스뿐만 아니라 이러한 벡터에 대해서도 어텐션 메커니즘을 실행하도록 선택할 수 있습니다. 그런 다음 이러한 벡터는 교육 과정에서 업데이트되고 추론 시 주의 메커니즘에서도 사용됩니다.

그런 다음 키-값 벡터는 일종의 색인화된 지식 기반 역할을 합니다. 변환기 언어 모델이 "2차 세계 대전이 끝난"이라는 문장의 다음 단어를 예측하려는 경우 2차 세계 대전이 끝난 연도를 묻는 것에 해당하는 다음 위치에 대한 쿼리가 있을 것입니다. 하지만, 이 정보는 이제 문장의 다른 곳 어디에서나 찾을 수 있으므로 모델은 추측해야 합니다.

그러나 전용 메모리 뱅크를 사용하면 변환기는 이와 같은 모든 종류의 작은 정보를 저장할 수 있으며 쿼리 벡터가 메모리 뱅크의 올바른 키에 도달하면 그 안의 올바른 정보에 액세스할 수 있습니다.

관심있는 분들을 위한 몇 가지 기술적 세부 사항: 메모리 벡터에 대한 위치 임베딩은 0이고 키와 값은 매트릭스에 쌓이고 어텐션 메커니즘을 실행하기 위한 시퀀스에 연결됩니다.

신경망의 전용 메모리 장치에 대한 아이디어는 완전히 새로운 것이 아닙니다. 그러나 이 어텐션 메커니즘을 흥미롭게 만드는 것은 상대적으로 단순한 신경망 아키텍처에 더 많은 매개변수를 주입하는 방법으로 영구 메모리 뱅크를 사용한다는 아이디어입니다.

### Product-Key Memory

적어도 딥 러닝 문헌에서 점점 더 인기를 얻고 있는 것 같기 때문에 메모리 증강 주의 집중(attention) 토끼 구멍에 대해 조금 더 자세히 살펴보겠습니다.

우리가 살펴볼 다음 주의 집중(attention) 메커니즘 + 메모리 장치는 그 자체로 보면 그다지 멋져 보이지 않습니다. 하지만 실제로는 이 레이어가 도입된 후에 나온 두 가지 최신 트랜스포머인 XLM 및 CTRL에서 사용됩니다.

이 논문이 출판되었을 때 트랜스포머의 메모리는 이미 중요했습니다. 그래서 이 프로젝트의 목표는 메모리를 더 효율적으로 만드는 것이었습니다.

그것은 우리가 이야기했던 이전 메모리 메커니즘과 매우 유사한 전제로 시작합니다. 키와 값이 모두 벡터인 키-값 쌍의 대규모 컬렉션으로 구성된 메모리 뱅크가 있습니다.


영구 메모리에서는 전체 메모리 뱅크를 관리하므로 메모리 뱅크가 너무 커지면 빠르게 폭발할 수 있습니다(지난 번에는 사용자가 피드포워드 네트워크에서 손실된 매개변수를 대체하기 위해 메모리를 사용하려고 했기 때문에 큰 문제는 아니었습니다). 여기서 저자는 다른 솔루션을 제안합니다.

각 쿼리에 거의 쓸모가 없는 거대한 메모리 뱅크를 사용하는 대신 몇 개의 키를 선택하고 해당 값을 사용하는 것이 어떻습니까? 구체적으로 그들은 그림 7-4와 같이 쿼리로 내적을 최대화하는 상위 k개의 키를 찾고 $k$ 해당 값의 가중 합계를 사용하여 메모리 뱅크에서 결과를 얻을 것을 제안합니다.

이것은 깔끔한 것처럼 보이지만 더 시원해집니다. 많은 수의 키가 있는 경우를 고려하십시오. 그러면 상위 키를 선택하기 전에 유사성 점수를 얻기 위해 각 키로 쿼리를 내적해야 하기 때문에 많은 내적을 계산해야 합니다.

top-$k$ 키 검색을 보다 효율적으로 만들기 위해 각 키를 반으로 나눌 수 있습니다. 예를 들어 10개의 요소가 있는 벡터/배열로 키를 갖는 대신 5개의 요소가 있는 2개의 키를 가질 수 있습니다. 각.

이제 오래된 학부생 조합론 교과서를 꺼내면 n 개의 반키가 있고 전체 키가 두 개의 반키의 연결이라고 생각하면 총 $n^2$ 키까지 만들 수 있음을 알 수 있습니다. 3개의 하위 키가 있는 예가 그림 7-5에 나와 있습니다. 이것이 의미하는 바는 $n^2$ 값 벡터의 메모리 뱅크에 필요한 것은 $n$ 하프 키뿐이라는 것입니다!

그림 7-6은 하프 키의 기능을 사용하여 이제 메모리 뱅크의 값에 액세스하는 방법을 보여줍니다.

이제 이 다이어그램이 말하는 내용을 분석해 보겠습니다. 먼저 쿼리 벡터를 두 부분으로 나눕니다. 쿼리 벡터의 전반부를 사용하여 절반 키의 모든 전반부를 내적하고 상위 $k$ 하위 키를 선택합니다. 아래쪽 절반 하위 키에 대해 동일한 작업을 수행합니다.

이제 전반부에 대해 $k$ 하위 키를 선택하고 하위 절반에 대해 $k$ 하위 키를 선택했으므로 선택할 수 있는 전체 키가 $k^2$가 됩니다.

이제 검색할 엄청난 수의 키 대신 $k^2$만 있으면 됩니다. 따라서 쿼리와 $k^2$ 키 간의 내적을 계산하지만 이번에는 전체 쿼리와 키를 사용합니다. 여기서 우리는 $k^2$가 전체 메모리 뱅크 크기보다 훨씬 작다고 가정하고 있으므로 전체 키를 모두 검색하는 것보다 실제로 훨씬 더 효율적입니다.

여기서부터는 표준 스케일 내적 어텐션 계산일 뿐입니다. 방금 선택한 $k$ 메모리 장치에 참석하면 변압기에 연결할 수 있는 매우 효율적인 메모리 모듈을 갖게 됩니다.

하지만 이 모든 노력이 그만한 가치가 있습니까? 이 하프 키 메모리 방법은 실제로 얼마나 잘 작동합니까? 음, 논문에 따르면 그들은 단지 12개 레이어 + 메모리를 사용하여 24개 레이어 BERT를 이길 수 있었습니다. 따라서 결정은 귀하에게 맡기겠습니다.

여기에서 우리는 흥미로운 전통적인 주의 집중(attention) 메커니즘에 대한 변형 세트를 제공했습니다. 그러나 이 목록은 결코 완전하지 않으며 관심(attention) 메커니즘에 대한 관심은 사상 최고입니다. "주의 집중(attention) 메커니즘" 및 "변압기"와 같은 용어에 대한 Google 트렌드 결과는 과학 문헌의 검색 빈도보다 상승장 동안의 주가와 훨씬 더 비슷해 보입니다.

트랜스포머 아키텍처의 인기가 폭발적으로 증가함에 따라 딥 러닝 연구 커뮤니티 전체가 이를 낙관하기로 결정했으며 2017년 원래 트랜스포머가 출시된 이후 이 분야에서는 다음을 약속하는 트랜스포머 아키텍처의 새로운 변형이 유입되었습니다. 더 효율적이고, 더 잘 확장되고, 더 낮은 메모리 비용 등이 있습니다. 오늘날 우리가 한 권의 책에 포함할 수 있는 것보다 더 많은 변압기와 같은 아키텍처가 있으며 새로운 아이디어가 매주 쏟아지고 있습니다.

바라건대 이것은 트랜스포머가 물리학의 기본 방정식처럼 돌담에 새겨질 하나의 모놀리식 모델이 아니라는 인상을 줄 수 있기를 바랍니다. 오늘날 우리가 셀 수 있는 것보다 더 많은 변종이 있고, 곧 셀 수 있는 것보다 더 많은 변종이 있을 것 같고, 우리가 명명할 수 있는 것보다 더 많은 변종이 곧 있을 것 같습니다. Linformer, Longformer, Reformer, Performser 및 Perceiver는 영어 어휘를 빠르게 잠식하고 있는 원래 Transformer 아키텍처의 많은 새로운 변형 중 일부에 불과합니다.

이 아키텍처 환경을 탐색하는 것은 어렵습니다. 많은 경우, 연구 논문은 특정 작업을 수행하는 데 빵을 얇게 썬 것 이후로 자신의 아이디어를 가장 좋은 것으로 평가 하지만 다른 작업은 완전히 무시할 수 있습니다. 예를 들어, 대규모 연구실은 종종 매우 높은 컴퓨팅 예산을 보유하고 있으며, 벤치마크 순위표에서 몇 퍼센트의 차이로 1위를 차지하기 위해 엄청나게 많은 컴퓨팅 리소스를 소비할 수 있는 새로운 아키텍처를 개발하는 데 집중합니다. 고맙게도 많은 연구자들은 이제 모든 사람이 침실 2개짜리 스튜디오 아파트에 슈퍼컴퓨팅 클러스터를 설치할 수 있는 것은 아니며 작고 가벼운 모델을 만드는 데 대한 관심이 높아지고 있음을 이해하고 인식하고 있습니다.

그러나 이와는 별개로 트랜스포머는 처음에는 잘 맞지 않는 것처럼 보일 수 있는 다른 영역에서 점점 더 많이 사용되고 있습니다. 그중 하나가 컴퓨터 비전입니다.

## Transformers for Computer Vision

## Conclusion

이제 트랜스포머 아키텍처에 대해 자세히 살펴보겠습니다. 이 책의 이 단계에서 오늘날 딥 러닝 연구자와 엔지니어가 생각하는 바에 대해 어느 정도 이해하기 시작했으면 합니다. 당신은 또한 자신의 아이디어를 생각해 내기 시작할 수도 있습니다. 일반적으로 이들은 다음과 같이 단순하게 시작합니다. "이봐, 이 어텐션 메커니즘은 내 GPU에 맞지 않아. 어떤 식으로든 더 작은 행렬 세트를 생성할 수 있을지 궁금하군." 우리는 당신이 이것을 시도하는 것이 좋습니다! 종종 이러한 단순한 아이디어는 반복과 테스트를 거쳐 사람들이 획기적인 연구 아이디어와 혁신적인 신제품을 만들도록 이끈다.

다음은 이 장의 핵심 아이디어에 대한 간략한 요약입니다.:

- 트랜스포머는 Vaswani 등의 2017년 논문 "Attention Is All You Need"에서 처음 제안 되었습니다.
- 트랜스포머는 RNN 아키텍처의 반복 부분을 제거하고 어텐션 메커니즘만 사용하여 문장 전체에서 병렬화할 수 있도록 합니다.
- 어텐션 메커니즘은 신경망에서 "전역적 특징"(큰 입력 시퀀스의 모든 지점으로부터의 정보)을 수집하고 결합할 수 있게 해주는 일종의 계층입니다.
- 어텐션 메커니즘은 다양한 형태로 제공되며 변환기뿐만 아니라 많은 도메인과 아키텍처에서 사용됩니다.
- Transformer 아키텍처에서 사용되는 표준 어텐션 메커니즘을 MHSA(Multi-Head Self-Attention)라고 합니다. 입력을 작은 키 공간으로 변환하고 내적 주의를 여러 번 반복합니다.
- 어텐션 메커니즘은 매우 강력하지만 계산 비용도 많이 듭니다. 표준 MHSA에는 $n^2$ 메모리 비용이 있습니다. 즉, 문장에 10개의 단어가 있는 경우 10*10 = 100 주의 가중치를 저장해야 합니다.
- $x$와 $y$ 사이의 주의 가중치는 "$x$와 $y$가 얼마나 관련이 있는가?"로 해석될 수 있습니다. 추상적인 의미에서(대명사 해결에 유용함).
- 주의 가중치는 유용한 시각화 도구가 될 수 있습니다.
- 새롭고 보다 계산적으로 효율적인 어텐션 메커니즘을 구축하는 방법을 평가하는 중요한 연구가 진행되고 있습니다. 지금까지는 명확한 최상의 접근 방식이 없으며 대부분의 실무자는 여전히 단순성을 위해 MHSA를 사용합니다.

이제 이 장이 끝나가고 있지만 트랜스포머의 이야기는 끝나지 않았습니다. 다음으로 지난 몇 년간 NLP의 폭발적인 성장을 이끈 일련의 사건을 살펴보겠습니다. 트랜스포머는 여기에서 큰 역할을 했으며 BERT, RoBERTa 및 GPT-3와 같은 새로운 모델은 어텐션 메커니즘에 대한 이 간단한 아이디어를 어떻게 확장하고 믿을 수 없을 정도로 강력한 NLP mdoel을 생성할 수 있는지 보여줄 것입니다.

# Fine-tuning BERT for sentiment analysis

## Contents
1. 학습데이터 확인
2. 보조함수 확인
3. SenitmentClassifier 확인
4. Analyser 확인
5. 학습 과정 확인
6. tests

## 학습 데이터

In [ ]:
!pip3 install torch
!pip3 install transformers
from typing import List, Tuple
from transformers import BertTokenizer, BertModel
import torch
from torch.nn import functional as F


DATA: List[Tuple[str, int]] = [
    # 긍정적인 문장 - 1
    ("난 너를 좋아해", 1),
    # --- 부정적인 문장 - 레이블 = 0
    ("난 너를 싫어해", 0)
]

TESTS = [
    "나는 자연어처리가 좋아",
    "나는 자연어처리가 싫어",
    "나는 너가 좋다",
    "너는 참 좋다",
]

## 보조함수

In [ ]:
# cuda를 사용할 수 있는지를 체크, 사용가능하다면 cuda로 설정된 device를 출력.
def load_device() -> torch.device:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return device


# 텐서를 구축하는 부분 - X
def build_X(sents: List[str], tokenizer: BertTokenizer, device: torch.device) -> torch.Tensor:
    """
    return X (N, 3, L).  N, 0, L -> input_ids / N, 1, L -> token_type_ids / N, 2, L -> attention_mask
    """
    encodings = tokenizer(text=sents,
                          add_special_tokens=True,
                          return_tensors='pt',
                          truncation=True,
                          padding=True)
    return torch.stack([
        encodings['input_ids'],
        encodings['token_type_ids'],
        encodings['attention_mask']
    ], dim=1).to(device)

# 텐서를 구축하는 부분 - y
def build_y(labels: List[int], device: torch.device) -> torch.Tensor:
    return torch.FloatTensor(labels).unsqueeze(-1).to(device)

## Sentiment Classifier

In [ ]:
class SentimentClassifier(torch.nn.Module):
    def __init__(self, bert: BertModel):
        super().__init__()
        self.bert = bert
        self.hidden_size = bert.config.hidden_size
        # TODO 1
        self.W_hy = torch.nn.Linear(..., ...)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X: (N, 3, L)
        :return: H_all (N, L, H)
        """
        input_ids = X[:, 0]
        token_type_ids = X[:, 1]
        attention_mask = X[:, 2]
        H_all = self.bert(input_ids, token_type_ids, attention_mask)[0]
        return H_all

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X:
        :return:
        """
        # TODO 2
        H_all = self.forward(X)
        H_cls = ...
        y_hat = ...  # (N, H) * (H, 1) -> (N, 1)
        return torch.sigmoid(y_hat)

    def training_step(self, X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        # TODO 3
        y_hat = self.predict(X)
        loss = ...
        return loss.sum()

## Analyser

In [ ]:

class Analyser:
    """
    Bert 기반 감성분석기.
    """
    def __init__(self, classifier: SentimentClassifier, tokenizer: BertTokenizer, device: torch.device):
        self.classifier = classifier
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, text: str) -> float:
        X = build_X(sents=[text], tokenizer=self.tokenizer, device=self.device)
        y_hat = self.classifier.predict(X)
        return y_hat.item()

## Training

In [ ]:
# 사전학습된 버트 모델을 로드
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
bert = BertModel.from_pretrained('beomi/kcbert-base')

# --- have a look at the config --- #
print(bert.config)
print(bert.config.hidden_size)

In [ ]:
# --- hyper parameters --- #
EPOCHS = 20
LR = 0.0001


device = load_device()
print(device)

# --- build the dataset --- # 
sents = [sent for sent, _ in DATA]
labels = [label for _, label in DATA]
X = build_X(sents, tokenizer, device)
y = build_y(labels, device)

# --- instantiate the classifier --- #
classifier = SentimentClassifier(bert)
classifier.to(device)  # 모델도 gpu에 올리기. 
optimizer = torch.optim.Adam(classifier.parameters(), lr=LR)  # 최적화 알고리즘을 선택.

# --- 학습시작 --- #
for epoch in range(EPOCHS):
    loss = classifier.training_step(X, y)
    loss.backward()  # 오차 역전파
    optimizer.step()  # 경사도 하강
    optimizer.zero_grad()  # 기울기 축적방지
    print(f"epoch:{epoch}, loss:{loss.item()} ")

## Test

In [ ]:
classifier.eval()
analyser = Analyser(classifier, tokenizer, device)

for sent in TESTS:
    print(sent, "->", analyser(sent))

## Conclusion